In [ ]:
#########################################
#               package                 #
#########################################
import os
import pyodbc
import pandas as pd
import numpy as np
import pymongo
from urllib.parse import quote_plus


In [ ]:
#########################################
#               Function                #
#########################################

class Mongo():

    ## Initialize  ##

    def __init__(self , user: str, password: str , host: str , port: str, database: str):
        self.user = user
        self.password = password
        self.host = host
        self.port = port 
        self.database = database

    ## Check Connection Status    
    def check_connect(self):
        try:
            with pymongo.Mongo

        return  connect_info

    def mysql_to_dataframe(self, query: str) -> pd.DataFrame:
        connect_info = self.check_connect()

        df = pd.read_sql(query, connect_info)

        return  df

    def create_table_sql(self, df : pd.DataFrame , table_name : str):
        column_list = list(df.columns)
        columns = [f"{col}" for col in column_list]
        type_dict = { 'object' : 'varchar(200)' ,
                      'int64' : 'int' ,
                      'int32' : 'int' ,
                      'float64' : 'float' ,
                      'datetime64[ns]' : 'datetime'}

        types = [type_dict[str(type)] for type in df.dtypes]
        sql = ""
        for col,type in zip(columns , types) :
            if col == column_list[-1] :
                sql += f"`{col}` {type}"
            else:
                sql += f"`{col}` {type},"
        sql = f""" CREATE TABLE {self.database}.{table_name} ({sql});"""

        return sql

    def insert_table_sql( self , df : pd.DataFrame , table_name : str ) :
        column_list = list(df.columns)
        columns = [f"{col}" for col in column_list]
        columns = ','.join(columns)
        sql = fr"INSERT INTO {self.database}.{table_name} ({columns}) VALUES "


        for row in df.itertuples(index = False , name = None) :

            row = list(row)
            remove_column_list = []
            remove_row_list = []

            for i in range(len(row)):
                try:
                    if np.isnan( row[i] ) :
                        remove_column_list.append(column_list[i])
                        remove_row_list.append(row[i])
                except:
                    pass

            for col in remove_column_list :
                column_list.remove(col)

            for r in remove_row_list :
                row.remove(r)

            row = tuple(row)

            if row == list(df.itertuples(index=False, name=None))[-1] :
                sql += f"""{row};"""
            else:
                sql += f"""{row},"""
        return sql


    def dataframe_to_mysql(self , df : pd.DataFrame , table_name : str) :
        connect_info = self.check_connect()
        cursor = connect_info.cursor()
        
        # with connect_info.cursor(as_dict = True) as cursor :
        try :
            
            sql = self.create_table_sql( df , table_name )
            cursor.execute(fr'{sql}')
            
            sql = self.insert_table_sql(df , table_name)
            cursor.execute(fr'{sql}')
            
            connect_info.commit() # 提交

            print('Create Table')
            
        except Exception as e :
            
            sql = self.insert_table_sql(df , table_name)
            cursor.execute(fr'{sql}')
            
            connect_info.commit() # 提交
            
            print('Table has already exist')
        finally :
            
            sql = self.insert_table_sql(df , table_name)
            cursor.execute(sql)
            
            connect_info.commit() # 提交